In [8]:
import librosa
import soundfile as sf
import numpy as np
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

In [9]:
def normalize_audio(filepath, target_rms=0.1):
    audio, sr = librosa.load(filepath, sr=None)
    rms = (audio ** 2).mean() ** 0.5
    scaling_factor = target_rms / rms
    normalized_audio = audio * scaling_factor
    sf.write(filepath, normalized_audio, sr)

In [10]:
def split_audio(file_path, segment_duration=6, sr=16000):
    y, _ = librosa.load(file_path, sr=sr)
    segment_length = segment_duration * sr
    segments = [y[i:i + segment_length] for i in range(0, len(y), segment_length)]
    return segments

In [11]:
def compute_spectrogram(signal, sr=16000, n_fft=2048, hop_length=512):
    # Compute the Short-Time Fourier Transform (STFT) to get the spectrogram
    spectrogram = librosa.stft(signal, n_fft=n_fft, hop_length=hop_length)
    # Convert the amplitude to decibels
    spectrogram_db = librosa.amplitude_to_db(np.abs(spectrogram), ref=np.max)
    # Transpose the spectrogram so that time is on the x-axis
    return spectrogram_db.T

def compare_spectrograms_dtw(spectrograms1, spectrograms2):
    dtw_distances = []
    for s1 in spectrograms1:
        for s2 in spectrograms2:
            # Calculate the DTW distance between s1 and s2
            distance, path = fastdtw(s1, s2, dist=euclidean)
            dtw_distances.append(distance)
    return dtw_distances


In [12]:
def find_best_matches(a_vs_b_similarities, a_vs_c_similarities,a_vs_a_similarities):
    best_matches = []
    for i in range(3):
        # Best match of the i-th segment of A with all segments of B
        start_idx_b = i * 3
        end_idx_b = (i + 1) * 3
        best_match_b = min(a_vs_b_similarities[start_idx_b:end_idx_b], default=-1)
        best_match_c = min(a_vs_c_similarities[start_idx_b:end_idx_b], default=-1)

        # Best match of the i-th segment of A with other segments of A
        start_idx_a = i * 3
        end_idx_a = (i + 1) * 3
        best_match_a = min([sim for j, sim in enumerate(a_vs_a_similarities[start_idx_a:end_idx_a]) if i != j], default=-1)

        # Determine if the best match is from B or C or from A
        best_match_source = 'B' if best_match_b < best_match_c else 'C'
        if best_match_source == 'B':
            best_match_source = 'B' if best_match_b < best_match_a else 'A'
        else:
            best_match_source = 'C' if best_match_c < best_match_a else 'A'
        best_matches.append((best_match_b,best_match_c ,best_match_a, best_match_source))
    
    return best_matches

In [15]:
pairs = [
    ("ID_1_level1__ɐ_.wav", "ID_1_level1__p_.wav", "ID_1_level1__b_.wav")
]

for file1, file2, file3 in pairs:
    # normalize_audio(file1)
    # normalize_audio(file2)
    # normalize_audio(file3)

    a_segments = split_audio(file1)
    b_segments = split_audio(file2)
    c_segments = split_audio(file3)

    a_spectrograms = [compute_spectrogram(segment) for segment in a_segments]
    b_spectrograms = [compute_spectrogram(segment) for segment in b_segments]
    c_spectrograms = [compute_spectrogram(segment) for segment in c_segments]

    a_vs_b_similarities_dtw_spec = compare_spectrograms_dtw(a_spectrograms, b_spectrograms)
    a_vs_c_similarities_dtw_spec = compare_spectrograms_dtw(a_spectrograms, c_spectrograms)
    a_vs_a_similarities_dtw_spec = compare_spectrograms_dtw(a_spectrograms, a_spectrograms)

    best_matches_dtw_spec = find_best_matches( a_vs_b_similarities_dtw_spec,a_vs_c_similarities_dtw_spec, a_vs_a_similarities_dtw_spec)

    # Output results
    for i, (best_match_b, best_match_c, best_match_a, best_match_source) in enumerate(best_matches_dtw_spec):
        print(f"Segment {i} of A:")
        print(f"  Best match with B: {best_match_b}")
        print(f"  Best match with C: {best_match_c}")
        print(f"  Best match with other A segments: {best_match_a}")
        print(f"  Best match source: {best_match_source}")

Segment 0 of A:
  Best match with B: 22225.088335499757
  Best match with B: 20083.69052912389
  Best match with other A segments: 17919.70614771594
  Best match source: A
Segment 1 of A:
  Best match with B: 20801.33429354625
  Best match with B: 17467.762998176917
  Best match with other A segments: 11365.949778373835
  Best match source: A
Segment 2 of A:
  Best match with B: 16252.331655686872
  Best match with B: 14358.021531980876
  Best match with other A segments: 11365.949778373835
  Best match source: A
